<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MC_Tiempo_AyC_vs_CRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Análisis de Eficiencia.**

El objetivo es comparar la eficiencia computacional de dos métodos de integración por Monte Carlo para estimar el valor de la integral en el intervalo [0,1] de la función: $$ g(x)=\sqrt{\arctan(x)} $$
Los métodos comoparados fueron:


1.   Acierto y Error (Hit-or-Miss)
2.   Monte Carlo Crudo (Mean Value)

La comparación se basó en una métrica de eficiencia que combina el tiempo de ejecución con la varianza de los resultados.



In [197]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time

In [198]:
def g(x):
  return np.sqrt(np.arctan(x))

In [199]:
n = 50000 # Puntos usados para la estimación. Se usará la misma n para ambos métodos

**1. Monte Carlo: Acierto y Error**

Para evaluar este método necesitábamos medir su consistencia y su costo computacional.

Creamos una función "envoltorio" `(evaluar_acierto_y_error)` que ejecutó la simulación principal `(estimacion_1)` un total de 50 veces.

In [200]:
def estimacion_1(n):
  aciertos=0
  for i in range (n):
    x = rd.random()
    y = rd.random()
    if y <= g(x):
      aciertos+=1
  I=aciertos/n
  return I

In [201]:
def evaluar_acierto_y_error(n_puntos, repeticiones=50):
  inicio_tiempo = time.time() # Es para iniciar el cronómetro antes del bucle
  lista_resultados = [] # Arreglo para guardar los 50 resultados

  for _ in range(repeticiones): # Para llamar a la función 50 veces
    resultado = estimacion_1(n_puntos)
    lista_resultados.append(resultado)

  fin_tiempo = time.time() # Detiene el tiempo al terminar el bucle
  tiempo_total = fin_tiempo - inicio_tiempo
  varianza_resultados = np.var(lista_resultados) # Calcula la varianza de todos los resultados obtenidos

  return tiempo_total, varianza_resultados

In [202]:
t_1, var_1 = evaluar_acierto_y_error(n, repeticiones=50)

In [203]:
print(f"Acierto y Error (50 ejecuciones):", "\n-----------------------------------")
print(f"Tiempo de ejecución (t_1): {t_1:.5f}s")
print(f"Varianza (var_Theta_1): {var_1:.8f}")

Acierto y Error (50 ejecuciones): 
-----------------------------------
Tiempo de ejecución (t_1): 6.76883s
Varianza (var_Theta_1): 0.00000304


**2. Monte Carlo: Crudo**

Se siguió un procedimiento idéntico para el segundo método.

Se implementó otra función envoltorio `(evaluar_crudo)` para ejecutar la simulación crudo 50 veces.


In [204]:
def crudo(n):
  G=[]
  for i in range (n):
    u=rd.random()
    G.append(g(u))
  return np.mean(G), np.var(G), np.std(G), G

Llamamos a la función 50 veces para calcular el tiempo:



In [205]:
def evaluar_crudo(n_puntos, repeticiones=50):
  inicio_tiempo = time.time()
  lista_de_medias = []

  for _ in range(repeticiones):
    resultado_completo = crudo(n_puntos) # La función devuelve una tupla
    estimacion_integral = resultado_completo[0] # De esa tupla, solo nos interesa el primer elemento [0]
    lista_de_medias.append(estimacion_integral)

  fin_tiempo = time.time() # Funciona igual que el anterior.
  tiempo_total = fin_tiempo - inicio_tiempo
  varianza_de_medias = np.var(lista_de_medias) # La guardamos en una lista

  return tiempo_total, varianza_de_medias


In [206]:
t_2, var_2 = evaluar_crudo(n, repeticiones=50)

In [207]:
print(f"Crudo (50 ejecuciones):", "\n-------------------------------------------")
print(f"Tiempo de ejecución (t_2): {t_2:.5f}s")
print(f"Varianza (var_Theta_2): {var_2:.8f}")

Crudo (50 ejecuciones): 
-------------------------------------------
Tiempo de ejecución (t_2): 5.80701s
Varianza (var_Theta_2): 0.00000091


**EFICIENCIA**

In [208]:
E = (t_1 * var_1) / (t_2 * var_2) # Fórmula de la eficiencia

print("--- Comparación: ---")
print(f"\nValores de Acierto y Error: t_1={t_1:.4f} s, var_1={var_1:.8f}")
print(f"Valores del método Crudo:   t_2={t_2:.4f} s, var_2={var_2:.8f}")

print(f"\nEficiencia calculada (E): {E:.4f}")
if E < 1:
  print("El método de Acierto y Error es más eficiente.")
elif E > 1:
  print("El método Crudo es más eficiente.")
else:
  print("Ambos métodos tienen una eficiencia equivalente.")

--- Comparación: ---

Valores de Acierto y Error: t_1=6.7688 s, var_1=0.00000304
Valores del método Crudo:   t_2=5.8070 s, var_2=0.00000091

Eficiencia calculada (E): 3.8802
El método Crudo es más eficiente.


Para la mayoría de las funciones de una dimensión, como la que analizamos, teóricamente se espera que el método Crudo sea más eficiente. Esto se debe a que el método de Acierto y Error "desperdicia" muchos cálculos en puntos que caen fuera del área de interés (los "misses"), mientras que el método Crudo aprovecha el 100% de los puntos generados. Por lo tanto, era más probable que el resultado del experimento fuera `E > 1`